In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import entropy
import datetime
import copy
import time
import wandb
import pickle as pkl
import sys
import math
sys.path.append('../../')
from data.processing import get_data
import models
from main.seir.fitting import single_fitting_cycle
from main.seir.forecast import get_forecast, forecast_all_trials, create_all_trials_csv, create_decile_csv_new
from main.seir.sensitivity import calculate_sensitivity_and_plot
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logging import log_wandb
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from viz.fit import plot_histogram, plot_all_histograms, plot_mean_variance, plot_scatter, plot_kl_divergence, plot_heatmap_distribution_sigmas, plot_all_params, plot_all_losses, plot_all_buckets, plot_cv_in_params, plot_recovery_loss, plot_confidence_interval
import yaml
from data.dataloader import SimulatedDataLoader

In [ ]:
def update_hosp_ratios(param_set, data_config, model_config):
    df = pd.read_csv(os.path.join('../../data/data/simulated_data/', data_config['output_file_name']), index_col=0)
    if model_config['end_date']:
        # print(model_config['end_date'])
        if isinstance(model_config['end_date'], int):
            if end_date > 0:
                raise ValueError('Please enter a negative value for end_date if entering an integer')
        if isinstance(model_config['end_date'], datetime.date):
            df['date'] = pd.to_datetime(df['date'])
            end_date = df.loc[df['date'].dt.date == model_config['end_date']].index[0] - len(df) + 1
    else:
        end_date = 0
    train_start_row = df.iloc[len(df) - (model_config['train_period'] + model_config['val_period'] + model_config['test_period']) + end_date]
    if data_config['model'] == 'SEIRHD':
        param_set['E_hosp_ratio'] = train_start_row['E'] / train_start_row['active']
        param_set['I_hosp_ratio'] = train_start_row['I'] / train_start_row['active']
    elif data_config['model'] == 'SEIRHD_Beta':
        param_set['E_hosp_ratio'] = train_start_row['E'] / train_start_row['active']
        param_set['I_hosp_ratio'] = train_start_row['I'] / train_start_row['active']
    elif data_config['model'] == 'SEIR_PU':
        param_set['E_hosp_ratio'] = train_start_row['E'] / train_start_row['active']
        param_set['I_hosp_ratio'] = train_start_row['I'] / train_start_row['active']
        param_set['Pu_pop_ratio'] = train_start_row['Pu'] / train_start_row['']
    return param_set

# Create short pickle file

In [ ]:
save_dir = '../../misc/predictions/exp3' 
file_name = 'exp3_fixed_params.pickle'
with open(os.path.join(save_dir, file_name), 'rb') as handle:
    input_dict = pkl.load(handle)

In [ ]:
output = []
for pred in input_dict['predictions_dicts']:
    pd_output = {}
    pd_output['prediction_dict'] = {}
    pd_output['prediction_dict']['best_params'] = pred['prediction_dict']['best_params']
    pd_output['prediction_dict']['df_loss'] = pred['prediction_dict']['df_loss']
    pd_output['run_tuple'] = pred['run_tuple']
    # pd['trials'] = pd['prediction_dict']['trials']
    # del pd['prediction_dict']
    output.append(pd_output)

In [ ]:
output_dict = {}
output_dict['predictions_dicts'] = output
output_dict['model_config'] = input_dict['model_config']
output_dict['val_periods'] = input_dict['val_periods']
output_dict['train_periods'] = input_dict['train_periods']
output_dict['end_dates'] = input_dict['end_dates']
with open(os.path.join(save_dir, "exp3_trial2_short.pickle"), 'wb') as handle:
    pkl.dump(output_dict, handle)

## Plots

In [ ]:
save_dir = '../../misc/predictions/exp3/short' 
file_name = 'exp3_trial1_short.pickle'
with open(os.path.join(save_dir, file_name), 'rb') as handle:
    run_dict = pkl.load(handle)

In [ ]:
model_config = run_dict['model_config']
simulated_config_filename = model_config['data']['dataloading_params']['config_file']
with open(os.path.join("../../configs/simulated_data/", simulated_config_filename)) as configfile:
    simulated_config = yaml.load(configfile, Loader=yaml.SafeLoader)    
actual_params = simulated_config['params']
del actual_params['N']

In [ ]:
train_periods = run_dict['train_periods']
end_dates = run_dict['end_dates'] 
format_str = '%d-%m-%Y' # The format

In [ ]:
plot_dict = {}
for param in actual_params :
    plot_dict[param] = {}
    for end_date in end_dates :
        end_date = datetime.datetime.strptime(end_date,format_str).date()
        plot_dict[param][end_date] = {}
        for train in train_periods : 
            plot_dict[param][end_date][train] = []

In [ ]:
def mape_loss(pred,true):
    return (abs(pred-true)/true)*100

def calc_loss(pred,true,method='mape'):
    if method == 'mape':
        return mape_loss(pred,true)

def compute_losses(best_params,true_params,method='mape'):
    losses = {}
    for param in best_params:
        losses[param] = calc_loss(best_params[param],true_params[param],method)
    return losses

In [ ]:
val_period = 7
for run in run_dict['predictions_dicts'] : 
    run_tuple = run['run_tuple']
    if(run_tuple['val'] != val_period):
        continue
    pred = run['prediction_dict']
    best_params = pred['best_params']
    model_config['split']['end_date'] = run_tuple['end_date']
    model_config['split']['train_period'] = run_tuple['train']
    model_config['split']['val_period'] = run_tuple['val']
    # print(model_config['split'])
    true_params = update_hosp_ratios(actual_params, simulated_config, model_config['split'])
    losses = compute_losses(best_params,true_params)
    print (run_tuple, best_params['beta'])
    for param in losses :
        plot_dict[param][run_tuple['end_date']][run_tuple['train']].append(losses[param])

In [ ]:
for param in actual_params : 
    for end_date in end_dates :
        end_date = datetime.datetime.strptime(end_date,format_str).date()
        for train in train_periods : 
            losses = np.array(plot_dict[param][end_date][train])
            plot_dict[param][end_date][train] = np.mean(losses)

In [ ]:
import seaborn as sn
cmap = sns.cm.rocket_r

n_subplots = len(plot_dict)
ncols = 2
nrows = math.ceil(n_subplots/ncols)
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, 
                        figsize=(18, 6*nrows))
ax_counter = 0
for param, param_dict in plot_dict.items():
    # if param in ['T_inf', 'T_inc', 'T_recov_fatal']:
        # continue
    ax = axs.flat[ax_counter]
    arr = []
    for end_date in end_dates:
        end_date_dict = param_dict[datetime.datetime.strptime(end_date,format_str).date()]
        l = []
        for train in train_periods:
            train_dict = end_date_dict[train]
            l.append(train_dict)
        arr.append(l)
    df_cm = pd.DataFrame(arr, index = end_dates,
                  columns = train_periods)
    sn.heatmap(df_cm, annot=False, ax=ax, cmap=cmap, cbar_kws={'label': 'recovery loss (MAPE %)'})
    ax_counter += 1
    ax.set_title(param)
    ax.set_xlabel("Train periods")
    ax.set_ylabel("End dates")